## ETF 데이터로 주가 방향 예측하기

- 랜덤 포레스트

- 부스팅

In [1]:
!pip install xgboost


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\jh102\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### 데이터

In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn import svm

import seaborn as sns; sns.set()

In [ ]:
# SPY = pd.read_csv('./SPY.csv',parse_dates=['Dates'], index_col = 'Dates')
# GLD = pd.read_csv('./GLD.csv',parse_dates=['Dates'], index_col = 'Dates')
# FXY = pd.read_csv('./FXY.csv',parse_dates=['Dates'], index_col = 'Dates')
# T10Y2Y = pd.read_csv('./T10Y2Y.csv',parse_dates=['Dates'], index_col = 'Dates')
# TED = pd.read_csv('./TED.csv',parse_dates=['Dates'], index_col = 'Dates')
# USO = pd.read_csv('./USO.csv',parse_dates=['Dates'], index_col = 'Dates')
# UUP = pd.read_csv('./UUP.csv',parse_dates=['Dates'], index_col = 'Dates')
# VIX = pd.read_csv('./VIX.csv',parse_dates=['Dates'], index_col = 'Dates')
# VWO = pd.read_csv('./VWO.csv',parse_dates=['Dates'], index_col = 'Dates')

In [ ]:
# t1 = SPY.merge(GLD, left_index=True,
#               right_index=True, how = 'outer',
#               suffixes=('_SPY', '_GLD'))
# list_df = ['FXY', 'T10Y2Y', 'TED', 'USO', 'UUP', 'VIX', 'VWO']
# for x in list_df:
#     t1 = t1.merge(globals()[x].add_suffix('_'+str(x)),
#                  left_index=True, right_index=True,
#                  how='outer')

In [ ]:
# df = t1.dropna()

In [ ]:
# df.tail()

In [ ]:
# df.head()

In [3]:
df = pd.read_csv('./ETFs_main.csv')
df

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.200,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.070,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.420,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220
3,2007-02-23,145.30,145.74,145.790,145.0,71962797.0,67.72,82.78,2.3809,0.31,50.46,25.04,10.58,40.035
4,2007-02-26,145.17,145.83,145.950,145.0,69320062.0,68.10,83.08,2.3795,0.31,50.90,25.04,11.15,39.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766,2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.180
2767,2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.870
2768,2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.320
2769,2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.900


In [ ]:
# pd.set_option('display.max_columns', 30)

In [4]:
print(os.getcwd())

c:\Users\jh102\OneDrive\바탕 화면\AIQuantTrading


### 기술적 지표 만들기

In [5]:
#기술적 지표 만들기
def moving_average(df, n):
    MA = pd.Series(df['CLOSE_SPY'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))
    df = df.join(MA)
    return df

def volume_moving_average(df, n):
    VMA = pd.Series(df['VOLUME'].rolling(n, min_periods=n).mean(), name='VMA_' + str(n))
    df = df.join(VMA)
    return df

def relative_strength_index(df, n):
    """Calculate Relative Strength Index(RSI) for given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    i = 0
    UpI = [0]
    DoI = [0]    
    while i + 1 <= df.index[-1]:
        UpMove = df.loc[i + 1, 'HIGH'] - df.loc[i, 'HIGH']
        DoMove = df.loc[i, 'LOW'] - df.loc[i + 1, 'LOW']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i = i + 1
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
    NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
    RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_' + str(n))
    df = df.join(RSI)
    return df

In [6]:
df = moving_average(df, 45) # biz day = 45
df = volume_moving_average(df, 45)
df = relative_strength_index(df, 14) # 시장강도지지수는 보통 14일이나 21일 사용

In [7]:
df = df.set_index('Dates')
df = df.dropna() #dropna하기 때문에 쿠션 데이터 추가해주면 좋음.
len(df)

2727

In [8]:
df['target'] = df['CLOSE_SPY'].pct_change()

In [9]:
df['target'].describe()

count    2726.000000
mean        0.000270
std         0.013026
min        -0.098448
25%        -0.004320
50%         0.000545
75%         0.005791
max         0.128249
Name: target, dtype: float64

### Case 1 : 수수료 0

In [10]:
# df['target'] = np.where(df['target'] > 0.0005 , 1, -1)
df['target'] = np.where(df['target'] > 0 , 1, -1)
df['target'].value_counts()

target
 1    1471
-1    1256
Name: count, dtype: int64

주가가 오른날이 1471일, 내린날이 1256일이다. 불균형 데이터가 아니다. cf) 신용카드 연체율은 9:1의 비율이 나올 것이다.

머신러닝 기반의 주가 트렌드 분류 모델은 54%~56% 정도의 정확도를 가진다면 매우 훌륭한 모델. 물론 AUC-ROC등의 지표와 MDD, 변동성 등 다 다양한 투자 성과와 지속가능 여부도 중요하다.

보통의 주가 트렌드 예측 모델은 54% ~56% 정도의 정확도 달성하기도 힘들기 때문에 트렌드가 불균형하면 잘못된 모델이 다올 수 있다.
ex) 모델이 훈련 중 특별한 패턴을 포착하지 못할 경우 정확도를 지표로 훈련하게 된다면, 모델 테스트를 할때 오른다고만 찍어도 54%의 확률로 맞는다.

따라서 최대한 균일한 트렌드를 갖도록 만들거나, 정확도가 아닌 다른 평가 지표를 사용해야 한다.

In [11]:
df['target'] = df['target'].shift(-1)

In [12]:
df = df.dropna()
len(df)

2726

In [13]:
df['target'] = df['target'].astype(np.int64)
y_var = df['target'] #-1과 1의 주가 하락과 상승의 정보 담은 변수
# 예측에 사용하지 않는 변수 삭제
x_var = df.drop(['target', 'OPEN','HIGH','LOW', 'VOLUME','CLOSE_SPY'], axis=1) 

In [14]:
x_var.head()

,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
Dates,,,,,,,,,,,
2007-04-27,67.56,83.7300,2.4474,0.55,51.84,24.54,12.45,41.750,143.551556,1.106696e+08,0.670018
2007-04-30,67.09,83.7166,2.4361,0.57,51.24,24.49,14.22,40.935,143.601556,1.116466e+08,0.531751
2007-05-02,66.66,83.3800,2.4366,0.59,49.59,24.66,13.08,42.020,143.680667,1.121613e+08,0.554050
2007-05-03,67.49,83.1100,2.4346,0.60,49.28,24.69,13.09,42.435,143.780222,1.123421e+08,0.601028
2007-05-04,68.19,83.2300,2.4006,0.60,48.30,24.60,12.91,42.595,143.905111,1.128853e+08,0.665987


In [15]:
up=df[df['target']==1].target.count()
total=df.target.count()
print('up/down ratio: {0:.2f}'.format((up/total)))

up/down ratio: 0.54


### 모델 학습

In [16]:
#훈련 세트와 테스트 세트를 나눈다.
#shuffle=False을 설정해 기간이 섞이지 않도록 만든다.
#기간을 정확히 설정하려면 기간별로 데이터 슬라이스 기법을 사용해 분할한다.
X_train, X_test, y_train, y_test = train_test_split(x_var, 
                                                    y_var, 
                                                    test_size=0.3, 
                                                    shuffle=False, 
                                                    random_state=3)

#훈련 세트와 테스트 세트에 포함된 양성 샘플(up) 비율을 확인한다.
train_count = y_train.count()
test_count = y_test.count()

print('train set label ratio')
print(y_train.value_counts()/train_count)
print('test set label ratio')
print(y_test.value_counts()/test_count)
# 훈련셋과 테스트셋의 양성/음성 비율이 비슷 --> 기간이 충분히 길다면 비슷하겠지만, 종목에 따라 다를 수 있음.

train set label ratio
target
 1    0.543501
-1    0.456499
Name: count, dtype: float64
test set label ratio
target
 1    0.530562
-1    0.469438
Name: count, dtype: float64


In [17]:
# 정확도, ROC-AUC 점수 등 혼동행렬을 계산하는 함수. micro-averge 내용 추가하여 수정.
def get_confusion_matrix(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    # precision = precision_score(y_test, pred)
    precision = precision_score(y_test, xgb_pred, average='micro')

    # recall = recall_score(y_test, pred)
    recall = recall_score(y_test, xgb_pred, average='micro')

    # f1 = f1_score(y_test, pred)
    f1 = f1_score(y_test, xgb_pred, average='micro')

    roc_score = roc_auc_score(y_test, pred)
    print('confusion matrix')
    print('accuracy:{0:.4f},precision:{1:.4f},recall:{2:.4f},F1:{3:.4f},ROC AUC score:{4:.4f}'.format(accuracy, precision, recall, f1,roc_score))

### 수익률이 0이였을때
1-1 번 모델(XGBoostClassifier)

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
# # LabelEncoder를 사용하여 클래스 변환
# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)

In [20]:
# xgb_dis = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
# xgb_dis.fit(X_train, y_train)
# # xgb_dis.fit(X_train, y_train_encoded)
# xgb_pred = xgb_dis.predict(X_test)

In [21]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split

# # 데이터를 학습 세트와 테스트 세트로 나누기
# X_train, X_test, y_train, y_test = train_test_split(x_var, 
#                                                     y_var, 
#                                                     test_size=0.3, 
#                                                     shuffle=False, 
#                                                     random_state=3)

# y_train을 [0, 1]로 변환
y_train_binary = (y_train + 1) // 2

# XGBClassifier 초기화 및 모델 학습
xgb_dis = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_dis.fit(X_train, y_train_binary)
xgb_pred = xgb_dis.predict(X_test)

In [22]:
xgb_dis

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [23]:
print(xgb_dis.score(X_train, y_train))

0.49580712788259956


In [24]:
get_confusion_matrix(y_test, xgb_pred)
# 매일 오른다고만 찍어도 54%의 정확도가 나와야 하는데, 훈련에서 49%정확도를 보여주고 테스트셋에 적용해보니 22.25%의 정확도가 나왔다.
# 과적합over-fitting 문제이다.

confusion matrix
accuracy:0.2225,precision:0.2225,recall:0.2225,F1:0.2225,ROC AUC score:0.4922


### 수익률이 0이였을 때
1-2 번 모델(Random Forest Classifier)

In [25]:
# 랜덤 포레스트 알고리즘. 매개변수를 지정하지 않고, GridSearchCV() 함수를 사용 --> 여러 가지 값을 돌아가면서 테스트
# n_estimators와 parameters 지정.
n_estimators = range(10,200,10)

params = {
    'bootstrap': [True],
    'n_estimators':n_estimators,
    'max_depth':[4,6,8,10,12],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_features':[4]
}

In [26]:
# 교차검증 방법
my_cv = TimeSeriesSplit(n_splits=5).split(X_train)

In [27]:
my_cv

<generator object TimeSeriesSplit.split at 0x0000022ECF26D180>

In [28]:
clf = GridSearchCV(RandomForestClassifier(), params, cv=my_cv, n_jobs=-1)

In [29]:
clf # page 209

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x0000022ECF26D180>,
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [4, 6, 8, 10, 12],
                         'max_features': [4], 'min_samples_leaf': [2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'n_estimators': range(10, 200, 10)})

xgboost를 훈련한 방법과 동일하게, RandomForestClassifier()에 설정한 파라미터와 교차검증 방법을 넣어주고 fit으로 훈련

In [30]:
import time

In [31]:
start_time = time.time()
clf.fit(X_train, y_train)
print(time.time() - start_time," seconds consumed")

953.7824938297272  seconds consumed


In [33]:
## Random Forest에서 best param
print('best parameter:\n', clf.best_params_) # 최적의 파라미터 나열
print('best prediction:{0:.4f}'.format(clf.best_score_)) # 가장 높았던 정확도 보여줌. 과적합은 아닌것 같지만 과소적합의 가능성.

best parameter:
 {'bootstrap': True, 'max_depth': 4, 'max_features': 4, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 30}
best prediction:0.5591


In [34]:
pred_con = clf.predict(X_test)
accuracy_con = accuracy_score(y_test, pred_con)
print('accuracy:{0:.4f}'.format(accuracy_con))
get_confusion_matrix(y_test, pred_con)
# 46%의 정확도 --> 여전히 마구잡이로 찍는 것보다 못한 수치.

accuracy:0.4694
confusion matrix
accuracy:0.4694,precision:0.2225,recall:0.2225,F1:0.2225,ROC AUC score:0.4892


### 수익률이 0.05% 이였을 떄

0.001% 처럼 미세한 움직임은 버리고 큰 폭으로 오르는 트렌드만 잡도록 훈련시켜보기

### 데이터

In [36]:
df = pd.read_csv('./ETFs_main.csv')
df = moving_average(df, 45)
df = volume_moving_average(df, 45)
df = relative_strength_index(df, 14)
df = df.set_index('Dates')
df = df.dropna()
df['target'] = df['CLOSE_SPY'].pct_change()

In [37]:
df['target'].describe() # SnP 500 지수는 평균적으로 0..02% 올랐고 / 표준편차는 1.3%이다. 매우 안정적이어서 머신러닝 알고리즘으로 패턴 잡아내기가 힘들 수 있음.

count    2726.000000
mean        0.000270
std         0.013026
min        -0.098448
25%        -0.004320
50%         0.000545
75%         0.005791
max         0.128249
Name: target, dtype: float64

In [38]:
df['target'] = np.where(df['target'] > 0.0005 , 1, -1)
# df['target'] = np.where(df['target'] > 0 , 1, -1)
df['target'].value_counts()

target
 1    1375
-1    1352
Name: count, dtype: int64

In [39]:
df['target'] = df['target'].shift(-1)
df = df.dropna()
len(df)

2726

In [40]:
df['target'] = df['target'].astype(np.int64)
y_var = df['target']
# 예측에 사용하지 않는 변수 삭제
x_var = df.drop(['target', 'OPEN','HIGH','LOW', 'VOLUME','CLOSE_SPY'], axis=1) 
up=df[df['target']==1].target.count()
total=df.target.count()
print('up/down ratio: {0:.2f}'.format((up/total)))

up/down ratio: 0.50


In [62]:
#훈련 세트와 테스트 세트를 나눈다.
#shuffle=False을 설정해 기간이 섞이지 않도록 만든다.
X_train, X_test, y_train, y_test = train_test_split(x_var, 
                                                    y_var, 
                                                    test_size=0.3, 
                                                    shuffle=False, 
                                                    random_state=3)

#훈련 세트와 테스트 세트에 포함된 양성 샘플(up) 비율을 확인한다.
train_count = y_train.count()
test_count = y_test.count()

print('train set label ratio')
print(y_train.value_counts()/train_count)
print('test set label ratio')
print(y_test.value_counts()/test_count)

train set label ratio
target
 1    0.509958
-1    0.490042
Name: count, dtype: float64
test set label ratio
target
-1    0.508557
 1    0.491443
Name: count, dtype: float64


### 수익률이 0.05% 이였을떄
2-1 번 모델 (XGBoost Classifier)

In [63]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# 클래스 레이블 수정
y_train = (y_train + 1) // 2
y_test = (y_test + 1) // 2

# XGBoost 모델 생성 및 학습
xgb_dis = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3, objective='binary:logistic')
xgb_dis.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [66]:
xgb_pred = xgb_dis.predict(X_test)
print(xgb_dis.score(X_train, y_train))
get_confusion_matrix(y_test, xgb_pred)

0.8579664570230608
confusion matrix
accuracy:0.5269,precision:0.5269,recall:0.5269,F1:0.5269,ROC AUC score:0.5241


이전 모델 보다는 안정적이고 과적합 문제가 적다.
보수적이라면 56% 이상의 정확도나 AUC-ROC 점수를 확보하는 것이 좋다. 단, 그럴 경우 과적합을 의심해 봐야 한다.

### 수익률이 0.05% 이였을 때
2-2번째 모델링(Random Forest Classifier)

In [67]:
n_estimators = range(10,200,10)

params = {
    'bootstrap': [True],
    'n_estimators':n_estimators,
    'max_depth':[4,6,8,10,12],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_features':[4]
}

In [68]:
# 교차 검증 방법으로 TimeSeriesSplit()을 사용.
my_cv = TimeSeriesSplit(n_splits=5).split(X_train)

In [69]:
clf = GridSearchCV(RandomForestClassifier(), params, cv=my_cv, n_jobs=-1)

In [70]:
start_time = time.time()
clf.fit(X_train, y_train)
print(time.time() - start_time," seconds consumed")

973.8514170646667  seconds consumed


In [71]:
## Random Forest에서 best param
print('best parameter:\n', clf.best_params_)
print('best prediction:{0:.4f}'.format(clf.best_score_))

best parameter:
 {'bootstrap': True, 'max_depth': 4, 'max_features': 4, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 40}
best prediction:0.5358


In [72]:
pred_con = clf.predict(X_test)
accuracy_con = accuracy_score(y_test, pred_con)
print('accuracy:{0:.4f}'.format(accuracy_con))
get_confusion_matrix(y_test, pred_con)

accuracy:0.5367
confusion matrix
accuracy:0.5367,precision:0.5269,recall:0.5269,F1:0.5269,ROC AUC score:0.5337


### 트리 계열 머신러닝 알고리즘 모델링에 도움될 하이퍼파라미터 설정법

- 학습률 learning_rate(LightGBM) / eta(XGboost)

    0~1의 값을 지정해 부스팅 스텝을 반복적으로 수행할 때 업데이트되는 학습률 값. 이전 트리의 오차를 얼마나 강하게 보정할 것인지를 제어. 일반적으로 0.01~0.02 값 사용

- 트리수 num_iterations(파이썬 래퍼 LightGBM) / n_estimators (XGboost, Random Forest)

    반복 수행하려는 트리의 개수를 지정. 트리가 많아질수록 성능이 좋아지지만 과적합의 위험도 올라감. LightGBM에서 디폴트 값으로 100이 설정됨. 큰 값으로 재설정. 랜덤 포레스트에서의 디폴트 값은 10

- 트리 깊이 max_depth(LightGBM, Xgboost, Random Frest)

    트리의 최대 깊이. 티폴트는 Xgboost에서 6이고 LightGBM에서는 -1로 깊이에 제한 없으나 너무 크면 과적합. 보통 3~10의 값 사용. 변수가 많을 경우 높게 설정. (얕은 트리는 분산은 작지만 편향이 많고, 깊은 트리는 편향은 적지만 분산이 크다.)

- 조기 종료 early stopping rounds(파이썬 래퍼 LightGMB에서는 early stopping round)

    검증 정확도가 더 이상 올라가지 않으 때, 지정된 학습 횟수 이전에 종료시키는 방법. 검증셋이 없다면 지정하지 않아도 됨.

- 서브 샘플링 bagging fraction (파이썬 래퍼 LightGBM) / subsample(Xgboost)

    일부 데이터만을 사용하는 bagging의 비율. 과적합을 제어하기 위한 수단. 0.5로 지정하면 절체 데이터의 절반을 트리 생성하는데 사용. 일반적으로 0.5~1의 값 사용.

- 변수 샘플링 feature fraction(파이썬 래퍼 LightGBM) / colsample_bytree(Xgboost)

    트리 생성에 필요한 변수(컬럼)를 샘플링 하는데 사용. 변수가 너무 많을 경우 과적합을 줄이는 방법으로 사용. 디폴트 값인 1로 설정하면 변수 샘플링은 하지 않음. 설정한다면 보통 0.7~0.9의 값 사용.

- 가중치 조절 min_sum_hessian_in_leaf(파이썬 래퍼 LightGBM) / min_child_weight

    과적합을 조절하는데 사용. 높은 값은 특정 샘플에 높은 의존도를 가지는 모델이 되는 것을 방지. 너무 높은 값을 설정하면 과소적합.

- 최대 리프 수 num_leaves(LightGBM)

    개별 트리가 가질 수 있는 최대 리프 수. LightGBM 모델의 복잡도를 제어하는 주요 파라미터. 트리가 매우 깊을 경우 num_leaves를 조정하면 효과.

- 감마 gamma(Xgboost)

    분할을 수행하는 데 필요한 최소 손실 감소를 지정. 값이 클수록 과적합 감소 효과

### 확장

1. 다중 분류 문제로 바꿔보기 ex) 상승과 하락 뿐 아니라, 보합, 3% 이상 상승 및 하락, 등

2. 트리기반 모델 뿐 아니라 SVM, 로지스틱 등 여러 알고리즘 써보기. 스케일에 민감한 알고리즘을 사용한다면 스케일 조정이 필요.

3. 변수를 바꿔보기

4. 예측하려는 종목/자산 바꿔보기. S&P 500이 아니라 변동성이 심한 USO지수에 대한 예측력이 좋을 수 있음.

5. 노이즈 제거 방법 사용.